In [ ]:
#bci ner +DWT+ ada

SyntaxError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner

/content/drive/.shortcut-targets-by-id/1PND44K5yRW8mjP-NB4mfwEivBcL8QvfE/2021_VIIT08_P300/Dataset/bci-ner


In [ ]:
!pip install -U imbalanced-learn
!pip install pyriemann

Requirement already up-to-date: imbalanced-learn in /usr/local/lib/python3.7/dist-packages (0.8.0)


In [ ]:
import pandas as pd
import random
from pyriemann.estimation import XdawnCovariances , Covariances
from pyriemann.tangentspace import TangentSpace
from os.path import isfile
import numpy as np
import h5py
from imblearn.over_sampling import RandomOverSampler

In [ ]:
X_train=np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/train_data.npy')
X_test=np.load('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/test_data.npy')
Y_test = np.reshape(pd.read_csv('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/true_labels.csv', header=None).values, 3400)

Y_train = np.load ('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/Y_train.npy')

In [ ]:
np.count_nonzero(Y_train == 0),np.count_nonzero(Y_train == 1)

(1590, 3850)

In [ ]:
pip install numpy matplotlib scipy numba scikit-learn mne PyWavelets pandas

In [ ]:
pip install mne-features

In [ ]:
from mne_features.feature_extraction import extract_features
from mne_features.univariate import get_univariate_funcs

In [ ]:

X_train.shape
X_train=np.reshape(X_train,(16*340, 56,140))

In [ ]:
X_train.shape

(5440, 56, 140)

In [ ]:
params1=['mean','variance','std']
new_X1=extract_features(X_train,128, params1 )
new_X1.shape

(5440, 168)

In [ ]:
params2=['higuchi_fd','hurst_exp']
new_X2=extract_features(X_train,128, params2 )
new_X2.shape

(5440, 112)

In [ ]:

params3=['spect_entropy','ptp_amp']
new_X3=extract_features(X_train,128, params3 )
new_X3.shape

(5440, 112)

In [ ]:
params4 =['wavelet_coef_energy']
new_x4 =extract_features(X_train,128, params4 )
new_x4.shape

(5440, 224)

In [ ]:
XC = XdawnCovariances(nfilter=4,estimator='oas',xdawn_estimator="oas") # our transformer
TS = TangentSpace(metric='riemann')

X_train_newX = XC.fit_transform(X_train,Y_train)
# X_train = fit_transform(train_data)
print("Xdawn Covariances", X_train_newX.shape)

Xdawn Covariances (5440, 16, 16)


In [ ]:
X_train_newX = TS.fit_transform(X_train_newX)
print("Tangent Spaces", X_train_newX.shape)

Tangent Spaces (5440, 136)


In [ ]:
boss= np.concatenate((new_X1,new_X3,new_X2,new_x4,X_train_newX),axis=1)


In [ ]:
boss.shape

(5440, 752)

In [ ]:
## Doing the same with X_text

In [ ]:
X_test.shape
X_test=X_test.reshape(10*340, 56,140)

In [ ]:
params1=['mean','variance','std']
new_X1=extract_features(X_test,128, params1 )
new_X1.shape

(3400, 168)

In [ ]:
params2=['higuchi_fd','hurst_exp']
new_X2=extract_features(X_test,128, params2 )
new_X2.shape

(3400, 112)

In [ ]:
params3=['spect_entropy','ptp_amp']
new_X3=extract_features(X_test,128, params3 )
new_X3.shape

(3400, 112)

In [ ]:
params4 =['wavelet_coef_energy']
new_x4 =extract_features(X_test,128, params4 )
new_x4.shape

(3400, 224)

In [ ]:
X_test = XC.transform(X_test)
# X_test = Covariances('oas').fit_transform(X_test)
X_test = TS.transform(X_test)


In [ ]:
X_test_new= np.concatenate((new_X1,new_X3,new_X2,new_x4, X_test),axis=1)


In [ ]:
np.save('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/Proposed Feature Extraction using mne/X_test', X_test_new)
np.save('/content/drive/MyDrive/2021_VIIT08_P300/Dataset/bci-ner/Proposed Feature Extraction using mne/X_train', boss)

### Now Let's Apply ONLY ADABOOST

In [ ]:
#Importing AdaBoost Classifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix


In [ ]:
#creating an AdaBoost instance
clf = AdaBoostClassifier(random_state=96)


In [ ]:
#training the model
clf.fit(boss,Y_train)


AdaBoostClassifier(random_state=96)

In [ ]:
y_preds=clf.predict(X_test_new)
cls = classification_report(Y_test,y_preds)
print(cls)
print("------------COnfusion Matrix---------------")
confusion_matrix(Y_test,y_preds)


              precision    recall  f1-score   support

           0       0.39      0.15      0.22       989
           1       0.72      0.90      0.80      2411

    accuracy                           0.68      3400
   macro avg       0.55      0.53      0.51      3400
weighted avg       0.62      0.68      0.63      3400

------------COnfusion Matrix---------------


array([[ 152,  837],
       [ 241, 2170]])

### AdaBoost with Optuna + Plots

In [ ]:
!pip install optuna

In [ ]:
import optuna 
from optuna import Trial, visualization
from optuna.samplers import TPESampler
# from a import XGBClassifier
from sklearn.model_selection import cross_val_score


In [ ]:
def objective(trial: Trial,X,y) -> float:
    
    # joblib.dump(study, 'study.pkl')
    
    # train_X,test_X,train_y,test_y = train_test_split(X, Y, test_size = 0.30,random_state = 101)

    param = {
                "n_estimators" : trial.suggest_int('n_estimators', 1, 100),
                # 'depth':trial.suggest_int('depth', 2, 25),
                
                'learning_rate':trial.suggest_loguniform('learning_rate',0.05,0.5),
                # 'colsample_bytree':trial.suggest_discrete_uniform('colsample_bytree',0.1,1,0.01),
                
            }
    model = AdaBoostClassifier(**param)

    model.fit(boss,Y_train)

    return cross_val_score(model,X_test_new,Y_test).mean()


In [ ]:
study = optuna.create_study(direction='maximize',sampler=TPESampler())
study.optimize(lambda trial : objective(trial,boss,Y_train),timeout=600)


[I 2021-06-07 07:25:46,336] A new study created in memory with name: no-name-8094ce30-d78a-4143-8904-5d46410621da
[I 2021-06-07 07:26:34,753] Trial 0 finished with value: 0.6364705882352941 and parameters: {'n_estimators': 30, 'learning_rate': 0.29875191543828306}. Best is trial 0 with value: 0.6364705882352941.
[I 2021-06-07 07:26:36,411] Trial 1 finished with value: 0.6858823529411765 and parameters: {'n_estimators': 1, 'learning_rate': 0.2872329779679853}. Best is trial 1 with value: 0.6858823529411765.
[I 2021-06-07 07:29:02,497] Trial 2 finished with value: 0.6247058823529412 and parameters: {'n_estimators': 91, 'learning_rate': 0.23208551647496786}. Best is trial 1 with value: 0.6858823529411765.
[I 2021-06-07 07:31:31,613] Trial 3 finished with value: 0.6497058823529411 and parameters: {'n_estimators': 93, 'learning_rate': 0.07232614575694889}. Best is trial 1 with value: 0.6858823529411765.
[I 2021-06-07 07:33:30,072] Trial 4 finished with value: 0.6291176470588236 and paramete

In [ ]:
print('Best trial: score {},\nparams {}'.format(study.best_trial.value,study.best_trial.params))
hist = study.trials_dataframe()
hist.head()


Best trial: score 0.6858823529411765,
params {'n_estimators': 1, 'learning_rate': 0.2872329779679853}


,number,value,datetime_start,datetime_complete,duration,params_learning_rate,params_n_estimators,state
0,0,0.636471,2021-06-07 07:25:46.343190,2021-06-07 07:26:34.753716,0 days 00:00:48.410526,0.298752,30,COMPLETE
1,1,0.685882,2021-06-07 07:26:34.755482,2021-06-07 07:26:36.410873,0 days 00:00:01.655391,0.287233,1,COMPLETE
2,2,0.624706,2021-06-07 07:26:36.412338,2021-06-07 07:29:02.497712,0 days 00:02:26.085374,0.232086,91,COMPLETE
3,3,0.649706,2021-06-07 07:29:02.499565,2021-06-07 07:31:31.612806,0 days 00:02:29.113241,0.072326,93,COMPLETE
4,4,0.629118,2021-06-07 07:31:31.614454,2021-06-07 07:33:30.072414,0 days 00:01:58.457960,0.237979,74,COMPLETE


In [ ]:
#creating an AdaBoost instance
clf = AdaBoostClassifier(**study.best_trial.params)


In [ ]:
#training the model
clf.fit(boss,Y_train)


AdaBoostClassifier(learning_rate=0.2872329779679853, n_estimators=1)

In [ ]:
y_preds=clf.predict(X_test_new)
cls = classification_report(Y_test,y_preds)
print(cls)
print("------------COnfusion Matrix---------------")
confusion_matrix(Y_test,y_preds)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       989
           1       0.71      1.00      0.83      2411

    accuracy                           0.71      3400
   macro avg       0.35      0.50      0.41      3400
weighted avg       0.50      0.71      0.59      3400

------------COnfusion Matrix---------------


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



array([[   0,  989],
       [   0, 2411]])

In [ ]:
clf.feature_importances_


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.